In [1]:
import sys
import os

# Get the directory of the current file
try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_dir = os.getcwd()  # Use current working directory in interactive environments

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Insert the parent directory into sys.path
sys.path.insert(0, parent_dir)

from models.mcnp_cell_factory import CellFactory


In [2]:
test_data = [
    ("999 0 -404:405:408           imp:n=0", 
     {"id": 999, "material_id": 0, "density": 0, "surfaces": [404, 405, 408], "importance": {"n": 0}}),
    ("3 0 (-3 1 2):(-2 -1)", 
     {"id": 3, "material_id": 0, "density": 0, "surfaces": [3, 1, 2, 2, 1], "importance": {"n": 0}}),
    ("12  81 -2.35 1 -3   -13 12 imp:n=1", 
     {"id": 12, "material_id": 81, "density": -2.35, "surfaces": [1, 3, 13, 12], "importance": {"n": 1}}),
    ("11 0 1 -2 imp:n=1", 
     {"id": 11, "material_id": 0, "density": 0, "surfaces": [1, 2], "importance": {"n": 1}}),
    ("11 0 1 -2 3 -4", 
     {"id": 11, "material_id": 0, "density": 0, "surfaces": [1, 2, 3, 4], "importance": {}}),
    ("14  81 -2.35 1 -3   -15 14 imp:n =1", 
     {"id": 14, "material_id": 81, "density": -2.35, "surfaces": [1, 3, 15, 14], "importance": {"n": 1}}),
    ("23 12   -9.024 (3 -4 101 -102) #2        imp:n =1 u=   4 vol =106.478", 
     {"id": 23, "material_id": 12, "density": -9.024, "surfaces": [3, 4, 101, 102], 
      "importance": {"n": 1}, "universe": 4, "volume": 106.478}),
    ("24 12   -9.024 (1: -2: 3:-4) #2        imp:n=1 u=4 vol =106.478", 
     {"id": 24, "material_id": 12, "density": -9.024, "surfaces": [1, 2, 3, 4], 
      "importance": {"n": 1}, "universe": 4, "volume": 106.478})
]

In [3]:
line = test_data[-2][0]
import re
pattern = re.compile(r'(\d+)\s+(\d+)\s+(\S+.*)?')
match = pattern.search(line)
print(CellFactory.create_from_match(match))
print(CellFactory.extract_keywords(match.group(3)))
cell = CellFactory.create_from_input_line(test_data[0][0])
print(cell)

Cell 23: Material ID 12, Surfaces [3, 4, 101, 102], Cells [2], Importance {'n': 1}, data cards {'vol': 106.478, 'u': 4}
('-9.024 (3 -4 101 -102) #2', {'vol': '106.478', 'u': '4', 'imp:n': '1'})
Cell 999: Material ID 0, Surfaces [404, 405, 408], Importance {'n': 0}, data cards {'vol': None, 'u': None}


# Testing Surfaces

In [4]:
from models.mcnp_input_cards import Surface
test_data = [
    ("1 px 0.0", {"id": 1, "surface_type": "px", "parameters": "0.0", "transformation": None, "comment": None}),
    ("2 py 1.0", {"id": 2, "surface_type": "py", "parameters": "1.0", "transformation": None, "comment": None}),
    ("3 pz -1.0", {"id": 3, "surface_type": "pz", "parameters": "-1.0", "transformation": None, "comment": None}),
    ("4 5 px 2.0", {"id": 4, "surface_type": "px", "parameters": "2.0", "transformation": "5", "comment": None}),
    ("5 px 3.0 $Test comment", {"id": 5, "surface_type": "px", "parameters": "3.0", "transformation": None, "comment": "Test comment"}),
]


In [5]:
surface = Surface.create_from_input_line(test_data[0][0])
line = test_data[0][0]
match = re.search(r'^\d+', line)
print(match.group(0))

1


In [6]:
print(surface)

Surface 1: px 0.0 
